<a href="https://colab.research.google.com/github/gabrieladamasceno/AM_Desafio6/blob/main/Desafio_06_Treinamento_de_modelos_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Desafio 06 - Treinamento de modelos CNN**

Professor: Cleber Zanchettin

Aluna: Maria Gabriela Lima Damasceno

# Introdução

O objetivo deste exercícido é utilizar um dataset do Kaggle que classifica frutas dos tipos maçã, banana e laranja em frutas frescas e podres. O modelo precisaria obter uma acurácia maior que 92%, sendo assim foi utilizada o método de Transfer learning, uma técnica do Deep learning que utiliza de um modelo pré-treinado para iniciar uma nova atividade. Neste projeto foi utilizado o modelo VGG16 (Rede Visual de Oxford 16) que é uma rede neural convolucional (CNN).

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Dados

##Dataset

Para utilizar o dataset de classificação das frutas [classificação das frutas](https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification) foram usadas funções que importam diretamente para o colab através da API do Kaggle, desta forma são criadas as pastas do dataset no colab, já dividas entre treino e validação.

In [2]:
#!pip install kaggle;

#Importa o token da API do Kaggle
from google.colab import files
files.upload() #enviar o arquivo kaggle.json

#Crias as pastas de armazenamento do dataset
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#Alterar a permissão para evitar o aviso durante a partida da ferramenta Kaggle
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

#Aceitar a competição e copiar o endereço da API para o download do dataset
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification

#Descompactar o arquivo baixado
!unzip \*.zip  && rm *.zip


A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.09 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.18 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.24 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.37 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.02.51 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.02 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.12 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.21 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 2018-06-07 at 3.03.31 PM.png  
  inflating: dataset/train/rottenapples/rotated_by_60_Screen Shot 20

##ImageNet

O VGG16 usou um conjunto de dados chamado ImageNet, que contém milhões de imagens rotuladas em mil categorias diferentes, para ser treinado inicialmente. Todas as camadas convolucionais do VGG16 usam filtros de convolução com tamanho 3x3.

In [3]:
# Usando o Imagenet
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


# Modelo

## Congelando as camadas do modelo base

 Por estar sendo usada a técnica de transfer learning é padrão congelar as camadas do modelo base, para manter os pesos que foram aprendidos durante o treinamento no conjunto de dados ImageNet.

In [4]:
# Congelando as camadas
base_model.trainable = False

## Adicionando camadas no modelo base

O primeiro passado após ser estabelecida a base do modelo é colocar os dados de entrada na dimensão certa do modelo.

Mais camadas são adicionadas ao modelo VGG16, que usa camadas Global Average Pooling (GAP) para reduzir a resolução espacial da representação e manter as características mais importantes.

Como o modelo tem o objetivo de fazer classificações multiclasse é preciso que o número de neurônios da camada Dense seja correspondente ao número de classes, nesta camada a ativação 'softmax' faz uma distribuição de probabilidade entre as classes.

Após isso o modelo é ciado com as entradas e saídas e é exibida sua sumarização.



In [5]:
# Colocando os dados de entrada na dimensão certa
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Camada Global Average Pooling (GAP)
x = keras.layers.GlobalAveragePooling2D()(x)

# Camada Dense
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combinação de entradas and saídas para criar o modelo
model = keras.Model(inputs, outputs)

In [6]:
# Sumário
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## Compilando o Modelo

Como trata-se de um problema de várias classes a função de perda CategoricalCrossentropy é apropriada para ser usada na compilação do modelo, juntamente com otimizador adam e com metas relacionadas a acurácia.

In [7]:
from tensorflow import keras

model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[keras.metrics.CategoricalAccuracy()])

## Aumentando os dados

Em datasets pequenos é comum que as imagens de treino sejam tratadas de forma que pareçam ser diferentes entre si, sendo roatacionadas, colocadas com zoom e etc para que sejam geradas mais amostras variadas para o treino do modelo, isso é feito através do ImageDataGenerator. Como no caso deste dataset haviam muitas amostras, foi preciso somente definir a média amostral como sendo zero.

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 # Define a média amostral como zero
datagen_train = ImageDataGenerator(samplewise_center=True)
datagen_valid = ImageDataGenerator(samplewise_center=True)

## Carregando o Dataset

O dataset já estava dividindo entre as pastas de treino e validação, então bastou colocá-los em suas respectivas variáveis e atribuir o tamanho da dimensão desejada, 224 x 224 pixels.

Foram indicadas que as classes eram do tipo RGB, tendo 3 canais com as cores vermelho, verde e azul e do tipo "categorical", que indica que as labels estão na classificação one-hot, ou seja, na forma binária.

In [9]:
# Carregando e iterando o dataset de treino
train_it = datagen_train.flow_from_directory(
    "/content/dataset/train",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)
# Carregando e iterando o dataset de validação
valid_it = datagen_valid.flow_from_directory(
    "/content/dataset/test",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


# Resultados

## Treinamento do Modelo

Os valores de treino e validação foram passados para afunção fit() do modelo que a cada 20 épocas, cada uma como 340 iterações, calculou o loss e a acurácia.

In [10]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5577: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


340/340 [==============================] - 106s 275ms/step - loss: 0.7985 - categorical_accuracy: 0.8060 - val_loss: 0.1990 - val_categorical_accuracy: 0.9351
Epoch 2/20
340/340 [==============================] - 88s 256ms/step - loss: 0.1336 - categorical_accuracy: 0.9543 - val_loss: 0.1297 - val_categorical_accuracy: 0.9577
Epoch 3/20
340/340 [==============================] - 75s 219ms/step - loss: 0.0751 - categorical_accuracy: 0.9750 - val_loss: 0.1077 - val_categorical_accuracy: 0.9611
Epoch 4/20
340/340 [==============================] - 76s 222ms/step - loss: 0.0494 - categorical_accuracy: 0.9848 - val_loss: 0.0651 - val_categorical_accuracy: 0.9752
Epoch 5/20
340/340 [==============================] - 75s 220ms/step - loss: 0.0346 - categorical_accuracy: 0.9897 - val_loss: 0.0554 - val_categorical_accuracy: 0.9815
Epoch 6/20
340/340 [==============================] - 75s 219ms/step - loss: 0.0275 - categorical_accuracy: 0.9915 - val_loss: 0.0737 - val_categorical_accuracy: 0.9

## Ajuste fino no modelo

Se o modelo não tivesse atigindo 92% de precisão de validação poderia ser usado um modelo com taxa de aprendizado muito baixa.

In [11]:
'''
# Congelando as camadas bases do modelo
base_model.trainable = True

# Compilar o modelo com baixa taxa de aprendizagem
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[keras.metrics.CategoricalAccuracy()])

# Treinando o modelo
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)
'''

"\n# Congelando as camadas bases do modelo\nbase_model.trainable = True\n\n# Compilar o modelo com baixa taxa de aprendizagem\nmodel.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),\n              loss=keras.losses.CategoricalCrossentropy(from_logits=True),\n              optimizer='adam',\n              metrics=[keras.metrics.CategoricalAccuracy()])\n\n# Treinando o modelo\nmodel.fit(train_it,\n          validation_data=valid_it,\n          steps_per_epoch=train_it.samples/train_it.batch_size,\n          validation_steps=valid_it.samples/valid_it.batch_size,\n          epochs=10)\n"

## Avaliação do Modelo

Por fim, através da função evaluate() é possível obter uma tupla, onde o primeiro valor é o loss e o segundo valor é a acurácia que, sendo maior que 92%, indica que o modelo está funcionando como o esperado.

In [12]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

84/84 [==============================] - 15s 179ms/step - loss: 0.0259 - categorical_accuracy: 0.9922


[0.02588879130780697, 0.9922164678573608]

# Conclusão

Através desse experimento foi possível constatar a utilidade do uso de modelos pré-treinos como o VGG16 e do seu conjunto de dados Imagenet. Sendo assim, o resultado do treinamento do modelo utilizando Transfer Learning e CNN saiu como o esperado, já que obteve uma acurácia de validação de 99,22%.